In [9]:
import numpy as np
import xroms
from glob import glob
import matplotlib.pyplot as plt
import gc
import xarray as xr
import pandas as pd
import xrft as xrft
import warnings
import sys
import xrft
# import seaborn as sns
sys.path.append("/homes/metogra/iufarias/FeedbackSubmeso/useful/")
import romspickle

from dask.diagnostics import ProgressBar
import dask
dask.config.set(**{'array.slicing.split_large_chunks': True})
from dask.distributed import Client 


import scipy.integrate as integ
warnings.filterwarnings("ignore")

In [4]:
client = Client(memory_limit='80GB',n_workers=42)

In [5]:
client.dashboard_link

'http://127.0.0.1:34115/status'

In [10]:
chunks={'time_counter':3, 'x_rho':1000, 'y_rho':1520,'s_rho':80,'s_w':80}
path='/data/pacific/lrenault/SASI/CROCO/'

varf=xr.open_mfdataset(path + 'FULL/SASI*2012-01*.nc',data_vars='minimal',chunks=chunks)
vart=xr.open_mfdataset(path + 'SMTH/SASI*2012-01*.nc',data_vars='minimal',chunks=chunks)






In [11]:
varx = varf.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v","x_w":"xi_w",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
df,gf=xroms.roms_dataset(varx,Vtransform=varx.Vtransform.data)

vary = vart.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
dt,gt=xroms.roms_dataset(vary,Vtransform=vary.Vtransform.data)

In [12]:
timer_h=pd.DatetimeIndex(dt.ocean_time)
timer_d=pd.DatetimeIndex(dt.ocean_time.groupby("ocean_time.dayofyear").mean())

In [24]:
w_full_interp=df.w.xroms.isoslice(np.linspace(-5,-70, 10))[:,5,120:-120,120:800]

In [25]:
%%time

nf=2
wdws='hann' #'flattop','hann'
wdw_cor=True
scl='density'

with ProgressBar():
    Wf=xrft.isotropic_power_spectrum(w_full_interp.drop(['nav_lat_rho','nav_lon_rho']),
                                      dim=['eta_rho','xi_rho'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)

CPU times: user 10.2 s, sys: 4.77 s, total: 15 s
Wall time: 1min 45s


In [31]:
Wf[0].compute()

KeyboardInterrupt: 

Task exception was never retrieved
future: <Task finished name='Task-12261' coro=<Client._gather.<locals>.wait() done, defined at /homes/metogra/iufarias/miniconda3/envs/XROMS/lib/python3.8/site-packages/distributed/client.py:2054> exception=AllExit()>
Traceback (most recent call last):
  File "/homes/metogra/iufarias/miniconda3/envs/XROMS/lib/python3.8/site-packages/distributed/client.py", line 2063, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-12260' coro=<Client._gather.<locals>.wait() done, defined at /homes/metogra/iufarias/miniconda3/envs/XROMS/lib/python3.8/site-packages/distributed/client.py:2054> exception=AllExit()>
Traceback (most recent call last):
  File "/homes/metogra/iufarias/miniconda3/envs/XROMS/lib/python3.8/site-packages/distributed/client.py", line 2063, in wait
    raise AllExit()
distributed.client.AllExit
